In [3]:
import requests
import pandas as pd
import csv
import json
import os
import time
import random

In [4]:
api_key = "KFLTLHTFNSZS1BBJ"

In [ ]:
CSV_URL = f'https://www.alphavantage.co/query?function=LISTING_STATUS&state=active&apikey={api_key}'

key_df = pd.DataFrame(columns=['Symbol'])

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list[1:]:
        symbol = row[0]
        temp = pd.DataFrame({'Symbol': symbol}, columns=['Symbol'], index=[0])
        key_df = pd.concat([key_df, temp], ignore_index=True)

key_df

In [4]:
import os
files = os.listdir("./yahoo_data")
list_symbols = [x.split(".")[0] for x in files]
print(list_symbols.index("CHH"))

1803


In [15]:
print("DUET" in list_symbols)

True


In [ ]:
import requests
import os
import time


def download_csv(symbol, max_retries=5, delay=60):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    filename = os.path.join(os.getcwd(), f"yahoo_data/{symbol}.csv")

    def url(
        x): return f"https://query{x}.finance.yahoo.com/v7/finance/download/{symbol}?period1=0&period2=1722319418&interval=1d&events=history"
    for attempt in range(max_retries):
        url_test = url(1)
        response = requests.get(url_test, headers=headers)

        if response.status_code == 200:
            with open(filename, "wb") as file:
                file.write(response.content)
            print(f"File downloaded and saved as {filename}")
            return
        elif response.status_code == 429:
            print(
                f"Rate limit exceeded. Retrying in {delay} seconds... (Attempt {attempt + 1}/{max_retries})")
            time.sleep(delay)
        else:
            url_test = url(2)
            response = requests.get(url_test, headers=headers)

            if response.status_code == 200:
                with open(filename, "wb") as file:
                    file.write(response.content)
                print(f"File downloaded and saved as {filename}")
                return
            elif response.status_code == 429:
                print(
                    f"Rate limit exceeded. Retrying in {delay} seconds... (Attempt {attempt + 1}/{max_retries})")
                time.sleep(delay)
            else:
                print(
                    f"Failed to download the file. Status code: {response.status_code}")
                break
    else:
        print(
            f"Exceeded maximum retries ({max_retries}). File could not be downloaded.")


# Filename to save the downloaded file in the current directory
for symbol in list_symbols:
    # Download the file
    download_csv(symbol)

In [1]:

from bs4 import BeautifulSoup
import os
import json

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import ChromiumOptions
import os
import json
from bs4 import BeautifulSoup
from dags.module.object_client import MinioClient


def extract_HTML_content(symbol):
    # URL
    url = f"https://finance.yahoo.com/quote/{symbol}/profile"

    # Webdriver setup
    options = webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    filename = os.path.join(f"yahoo_data_info/{symbol}.html")

    # Minio client
    minio_client = MinioClient(
        "localhost", "9000", access_key="minio", secret_key="minio123")

    # Request
    driver.get(url)
    page_text = driver.find_element(
        By.XPATH, "/html/body/div[1]/main/section/section/section").get_attribute("innerHTML")

    # Write into a HTML file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(page_text)

    minio_client.put_object(bucket_name="bronze",
                            obj_name=filename, obj_file=filename)
    print(f"Successfully loaded {filename} into Minio.")
    driver.close()

In [20]:
for symbol in list_symbols:
    extract_HTML_content(symbol)
    time.sleep(10)

Successfully loaded yahoo_data_info/A.html into Minio.
Successfully loaded yahoo_data_info/AA.html into Minio.
Successfully loaded yahoo_data_info/AAA.html into Minio.
Successfully loaded yahoo_data_info/AAAU.html into Minio.
Successfully loaded yahoo_data_info/AACG.html into Minio.
Successfully loaded yahoo_data_info/AACI.html into Minio.
Successfully loaded yahoo_data_info/AACIU.html into Minio.
Successfully loaded yahoo_data_info/AACIW.html into Minio.
Successfully loaded yahoo_data_info/AACT.html into Minio.
Successfully loaded yahoo_data_info/AADI.html into Minio.
Successfully loaded yahoo_data_info/AADR.html into Minio.
Successfully loaded yahoo_data_info/AAGR.html into Minio.
Successfully loaded yahoo_data_info/AAGRW.html into Minio.
Successfully loaded yahoo_data_info/AAL.html into Minio.
Successfully loaded yahoo_data_info/AAMC.html into Minio.
Successfully loaded yahoo_data_info/AAME.html into Minio.
Successfully loaded yahoo_data_info/AAN.html into Minio.
Successfully loaded

In [14]:
from bs4 import BeautifulSoup

with open("A.html", "r", encoding='utf-8') as f:
    html_data = f.read()

soup = BeautifulSoup(html_data, "html.parser")

'<article class="gridLayout yf-85yv0j">\n <section class="container yf-ezk9pj" data-testid="quote-hdr">\n  <div class="top yf-ezk9pj">\n   <span class="exchange yf-1fo0o81">\n    <span>\n     NYSE - Delayed Quote\n    </span>\n    <i aria-hidden="true" data-svelte-h="svelte-sg6bkb">\n     â€¢\n    </i>\n    <span>\n     USD\n    </span>\n   </span>\n   <div class="left yf-ezk9pj wrap">\n    <section class="container yf-3a2v0c paddingRight">\n     <h1 class="yf-3a2v0c">\n      Agilent Technologies, Inc. (A)\n     </h1>\n    </section>\n    <div class="btns yf-ezk9pj">\n     <div class="menuContainer tw-pt-[3px] tw-pb-[2px] yf-1j5x891">\n      <button aria-controls="menu-15" aria-haspopup="true" class="follow-btn fin-size-x-small menuBtn rounded yf-122t2xs" data-rapid_p="13" data-testid="add-to-following" data-v9y="1" data-ylk="elmt:ticker;itc:1;cid:A;elm:intent-follow;sec:qsp-following;slk:menu-open;ticker:A">\n       <div class="icon fin-icon inherit-icn sz-medium yf-7v4gbg">\n        

In [18]:
# URL
url = "https://finance.yahoo.com/quote/A/profile"

# Webdriver setup
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

# Request
driver.get(url)
content = driver.find_element(By.XPATH, "/html/body/div[1]/main/section/section/section/article/section[2]/section[1]/div[1]")
display(content.text)
driver.close()

'Amounts are as of December 31, 2023 and compensation values are for the last fiscal year ending on that date. Pay is salary, bonuses, etc. Exercised is the value of options exercised during the fiscal year. Currency in USD.'

In [1]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from dotenv import dotenv_values
import json
import time
from dags.module.object_client import MinioClient


config = dotenv_values("dags/.env")
access_key = config['MINIO_ROOT_USER']
secret_key = config['MINIO_ROOT_PASSWORD']

def extract_symbol_list():
    # Arguments for the function
    minio_client = MinioClient("localhost", "9000", access_key=access_key, secret_key=secret_key)
    list_objects = minio_client.list_objects(bucket_name="symbol")
    list_symbols = [x.split(".")[0] for x in list_objects]
    return list_symbols[:1000]

extract_symbol_list()

['A',
 'AA',
 'AAA',
 'AAAU',
 'AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AACT',
 'AADI',
 'AADR',
 'AAGR',
 'AAGRW',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPB',
 'AAPD',
 'AAPL',
 'AAPR',
 'AAPU',
 'AAPX',
 'AAPY',
 'AAT',
 'AAXJ',
 'AB',
 'ABAT',
 'ABBV',
 'ABCB',
 'ABCL',
 'ABCS',
 'ABEO',
 'ABEQ',
 'ABEV',
 'ABG',
 'ABIO',
 'ABL',
 'ABLLL',
 'ABLLW',
 'ABLV',
 'ABLVW',
 'ABM',
 'ABNB',
 'ABNY',
 'ABOS',
 'ABR',
 'ABSI',
 'ABT',
 'ABTS',
 'ABUS',
 'ABVC',
 'ABVE',
 'ABVEW',
 'ABVX',
 'AC',
 'ACA',
 'ACAB',
 'ACABU',
 'ACABW',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACB',
 'ACCD',
 'ACCO',
 'ACDC',
 'ACEL',
 'ACES',
 'ACET',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACHC',
 'ACHL',
 'ACHR',
 'ACHV',
 'ACI',
 'ACIC',
 'ACIO',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACNT',
 'ACON',
 'ACONW',
 'ACP',
 'ACR',
 'ACRE',
 'ACRS',
 'ACRV',
 'ACSI',
 'ACST',
 'ACT',
 'ACTG',
 'ACTV',
 'ACU',
 'ACV',
 'ACVA',
 'ACVF',
 'ACWI',
 'ACWV',
 'ACWX',
 'ACXP'